# n-gram 유사도

In [3]:
from konlpy.tag import Komoran

# 어절 단위 n-gram
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x: x + num_gram] for x in range(0, len(text))] 
    return tuple(ngrams) 
#어절 단위로 n-gram 토큰을 추출해서 tuple형태로 저장함 슬라이싱을 이용해 n개씩 끊어서 토큰 저장

#유사도 계산
def similarity(doc1, doc2):
    cnt =0
    for token in doc1:
        if token in doc2:
            cnt +=1
    return cnt/len(doc1)
#doc1, doc2 토큰이 얼마나 동일한지 횟수를 카운트함

#문장 정의
sen1 ='6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sen2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sen3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

In [4]:
#형태소 분석기에서 명사(단어) 추출
komoran = Komoran()

bow1 = komoran.nouns(sen1)
bow2 = komoran.nouns(sen2)
bow3 = komoran.nouns(sen3)

In [5]:
# 단어 n-gram 토큰 추출
doc1 = word_ngram(bow1, 2) # 2-gram 방식으로 추출 n=2
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

In [8]:
#추출된 n-gram 토큰 출력
print(doc1)
print(doc2)
print(doc3)

(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
(('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'), ('님과 함께',))


In [9]:
# 유사도 계산
r1 = similarity(doc1, doc2)
r2= similarity(doc3, doc1)

#계산된 유사도 출력
print(r1)
print(r2)

0.6666666666666666
0.0


## 코사인 유사도 계산

In [10]:
from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm

#코사인 유사도 계산
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

#TDM 만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}
    
    for word in word_dics:
        freq_mat[word]= 0
    
    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word]+=1
    return freq_mat
#비교 문장에서 추출한 단어 사전을 기준으로 문장에 해당 단어들이 얼마나 포함되어 있는지
#나타내는 단어 문서 행렬(tdm)을 만드는 함수

#단어 벡터 만들기
def make_vector(tdm):
    vec =[]
    for key in tdm:
        vec.append(tdm[key])
    return vec
#단어 문서 행렬에서 표현된 토큰들의 출현 빈도 데이터를 벡터로 만들어주는 함수

In [16]:
#문장 정의
sen1 ='6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sen2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sen3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

In [17]:
#형태소 분석기에서 명사(단어) 묶음 리스트 생성
komoran = Komoran()

bow1 = komoran.nouns(sen1)
bow2 = komoran.nouns(sen2)
bow3 = komoran.nouns(sen3)

In [21]:
#단어 묶음 리스트를 하나로 합침
bow = bow1 + bow2 + bow3

# 단어 묶음에서 중복을 제거해 단어 사전 구축
word_dics =[]
for token in bow:
    if token not in word_dics:
        word_dics.append(token)
        

In [22]:
#문장별 단어 문서 행렬 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1, '선생': 1, '님과 함께': 1}


In [23]:
#문장 벡터 생성
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))

#코사인 유사도 계산
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc3, doc1)

print(r1)
print(r2)

0.8333333333333335
0.20412414523193154
